## Import Libraries

In [1]:
import pandas as pd
import googlemaps
import gmaps
import gmaps.datasets
import gmaps.geojson_geometries
import matplotlib.pyplot as plt

## Access Keys

In [2]:
f = open('gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

# Set up Gmaps API client
gmaps.configure(api_key=my_api_key)

# Set up Google Maps API client
gms = googlemaps.Client(key=my_api_key)

## Import dataset

In [3]:
df = pd.read_csv(r'Border_Crossing_Entry_Data.csv')

In [4]:
df.sample(10)

,Port Name,State,Port Code,Border,Date,Measure,Value,Latitude,Longitude,Point
335449,Baudette,Minnesota,3424,US-Canada Border,Oct 1998,Trucks,412,48.719,-94.590,POINT (-94.590467 48.719109)
253031,Turner,Montana,3306,US-Canada Border,Jun 2005,Trucks,20,49.000,-108.388,POINT (-108.387916 48.999527)
188369,Laredo,Texas,2304,US-Mexico Border,Apr 2009,Rail Containers Loaded,10065,27.500,-99.507,POINT (-99.507412 27.499561)
9525,International Falls,Minnesota,3604,US-Canada Border,Dec 2021,Personal Vehicle Passengers,6152,48.608,-93.401,POINT (-93.401355 48.6078)
96092,Richford,Vermont,203,US-Canada Border,Apr 2014,Personal Vehicle Passengers,17124,45.012,-72.589,POINT (-72.588559 45.01174)
85146,Whitlash,Montana,3321,US-Canada Border,May 2015,Personal Vehicle Passengers,107,48.997,-111.258,POINT (-111.257916 48.99725)
368959,Laurier,Washington,3016,US-Canada Border,Feb 1997,Bus Passengers,399,49.000,-118.224,POINT (-118.223777 49.000083)
301270,Point Roberts,Washington,3017,US-Canada Border,Aug 2003,Train Passengers,0,49.002,-123.068,POINT (-123.068055556 49.0020555547)
79807,Sherwood,North Dakota,3414,US-Canada Border,Sep 2015,Personal Vehicles,1496,48.999,-101.628,POINT (-101.627527 48.999305)
313098,Sasabe,Arizona,2606,US-Mexico Border,Jul 2001,Trucks,135,31.483,-111.544,POINT (-111.544363 31.483039)


In [5]:
df.shape

(390305, 10)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390305 entries, 0 to 390304
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Port Name  390305 non-null  object 
 1   State      390305 non-null  object 
 2   Port Code  390305 non-null  int64  
 3   Border     390305 non-null  object 
 4   Date       390305 non-null  object 
 5   Measure    390305 non-null  object 
 6   Value      390305 non-null  int64  
 7   Latitude   390305 non-null  float64
 8   Longitude  390305 non-null  float64
 9   Point      390305 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 29.8+ MB


In [7]:
df.describe()

,Port Code,Value,Latitude,Longitude
count,390305.000000,3.903050e+05,390305.000000,390305.000000
mean,2447.417522,2.868126e+04,43.982557,-99.745781
std,1203.813604,1.496351e+05,8.149411,18.255523
min,101.000000,0.000000e+00,25.952000,-141.001000
25%,2304.000000,0.000000e+00,42.999000,-114.728000
50%,3012.000000,1.370000e+02,48.496000,-101.628000
75%,3401.000000,2.912000e+03,49.000000,-89.585000
max,3814.000000,4.447374e+06,62.615000,-66.980000


In [8]:
df_markers = df[['Latitude','Longitude']]
df_markers

,Latitude,Longitude
0,26.404,-99.019
1,29.327,-100.928
2,49.000,-109.731
3,48.997,-111.258
4,31.673,-106.335
...,...,...
390300,43.659,-70.243
390301,44.992,-74.308
390302,48.999,-104.574
390303,49.000,-101.017


In [10]:
fig = gmaps.figure(center=(37.0902, -95.7129), zoom_level=4)

# Add Heatmap layer
heatmap_layer = gmaps.heatmap_layer(df_markers)
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
gmaps.figure(center=(37.0902, -95.7129), zoom_level=4)

Figure(layout=FigureLayout(height='420px'))

<Figure size 640x480 with 0 Axes>